In [14]:
import re
import scipy.stats as stats
import pandas as pd

In [13]:
with open("recalibrated_variants.postCGP.Gfiltered.Optom_subset.vcf", 'rt') as first_file:
    optom = {}
    info = []
    # create a list of locations in optom
    snp = {}
    for line in first_file:
        if line.startswith('#'):
            continue
        row = line.split('\t')
        ### remove the chr from the chromosomes
        tmp = [stuff.replace("chr", "") for stuff in row]
        ### create an index which is chr:bp
        chr_bp = tmp[0:2]
        index = ':'.join(map(str, chr_bp))
        ### output index, rsnumber, a1, a2, info
        info = [ tmp[2], tmp[3], tmp[4], tmp[6]]
        info.extend(tmp[7].split(';')[0:3])
        #if tmp[7].split(';')[3] != "set=FilteredInAll" :
        #    if tmp[9].split(':')[3] != "lowGQ" : 
        if index not in optom:
            optom[index] = info
        if tmp[2] not in snp and tmp[2] != '.':
            snp[tmp[2]] = 1


In [14]:
len(optom)

6084320

In [15]:
optom['4:144938033']

['rs5862681', 'A', 'AT', 'PASS', 'AC=30', 'AF=0.375', 'AN=30']

In [16]:
optom_all = pd.DataFrame(optom).T.reset_index()
optom_all.rename(columns={'index': 'chr:bp', 0: 'rs_id', 1:'A1', 2:'A2', 3:"optom_flag", 4:"AC", 5:"AF", 6:"AN"}, inplace=True)
optom_all.head()

,chr:bp,rs_id,A1,A2,optom_flag,AC,AF,AN
0,10:100000625,rs7899632,A,G,PASS,AC=0,AF=0.025,AN=30
1,10:100000645,rs61875309,A,C,PASS,AC=0,AF=0.013,AN=30
2,10:100001277,.,G,T,PASS,AC=0,AF=0.013,AN=30
3,10:100001507,.,A,C,PASS,AC=0,AF=0.013,AN=30
4,10:100002399,rs8181398,A,G,PASS,AC=30,AF=1.00,AN=30


In [17]:
optom_all.to_csv('optom_data_vsqr.csv', index=False, header=True, sep='\t')

## Get optom genos

In [26]:
with open("recalibrated_variants.postCGP.Gfiltered.Optom_subset.vcf", 'rt') as f:
    optom_geno = {}
    info = []
    # create a list of locations in optom
    for line in f:
        if line.startswith('#'):
            continue
        row = line.split('\t')
        tmp = [stuff.replace("chr", "") for stuff in row]
        ### create an index which is chr:bp
        chr_bp = tmp[0:2]
        index = ':'.join(map(str, chr_bp))
        if index not in optom_geno:
            homo_major = 0
            hets = 0
            homo_minor = 0
            lowGQ_count = 0
            for x in range(9,24) :
                data = tmp[x].split(':')
                #print(data)
                if data[0] == "0/0" :
                    homo_major += 1
                elif data[0] == "0/1" :
                    hets += 1
                elif data[0] == "1/1":
                    homo_minor += 1
                if len(data) > 3:
                    if data[3] == 'lowGQ':
                        lowGQ_count += 1
            info = [tmp[2], tmp[3], tmp[4], homo_major, hets, homo_minor, lowGQ_count]
            optom_geno[index] = info 

In [29]:
optom_geno['4:144938033']

['rs5862681', 'A', 'AT', 0, 0, 15, 4]

In [30]:
genos = pd.DataFrame(optom_geno).T.reset_index()
genos.rename(columns={'index': 'chr:bp', 0: 'rs_id', 1:'A1', 2:'A2', 3:'homo_major', 4:'hets', 
                      5:'homo_minor', 6:'lowGQ_numbers' }, inplace=True)
genos.head()

,chr:bp,rs_id,A1,A2,homo_major,hets,homo_minor,lowGQ_numbers
0,10:100000625,rs7899632,A,G,15,0,0,15
1,10:100000645,rs61875309,A,C,15,0,0,15
2,10:100001277,.,G,T,15,0,0,15
3,10:100001507,.,A,C,15,0,0,15
4,10:100002399,rs8181398,A,G,0,0,15,0


In [31]:
genos.to_csv('optom_genotpyes_vsqr.csv', index=False, header=True, sep='\t')

# Association with ALL populations

In [5]:
# Allele counts to keep
list_names =["AC", "AN", "AC_NFE", "AN_NFE", "AF"]
pattern = re.compile("|".join(list_names))

#not_pass_qc_gnomad = {}

with open("gnomad.exomes.r2.0.2.sites.vcf", 'rt') as EXAC:
    gnomad = {}
    for line in EXAC:
        if line.startswith('#'):
            continue
        row = line.split('\t')
        ### create an index which is chr:bp
        chr_bp = row[0:2]
        index = ':'.join(map(str, chr_bp))
        ### keep only those that are in optom:
        if index in optom:
            #if row[6] != "PASS":
            #    if index not in not_pass_qc_gnomad:
            #        not_pass_qc_gnomad[index] = [ row[2], row[6] ]
            #    continue
            ### output index, rsnumber, a1, a2, info
            output = [ row[2], row[3], row[4], row[6], row[-1].split(';')[-1] ]
            info = row[7].split(';')
            keep_info = [item for item in info if re.match(pattern, item)]
            output.extend(keep_info)
            #print(info)
            if index not in gnomad:
                gnomad[index] = output
        elif row[2] in snp:
            if index not in gnomad:
                ### output index, rsnumber, a1, a2, info
                output = [ index, row[2], row[3], row[4], row[6], row[-1].split(';')[-1] ]
                info = row[7].split(';')
                keep_info = [item for item in info if re.match(pattern, item)]
                output.extend(keep_info)
                gnomad[index] = output

In [6]:
len(gnomad)

188436

In [7]:
gnomad['5:172745269']

['rs4041247',
 'AAGAGAG',
 'AAGAGAGAG,AAGAGAGAGAG,AAGAG,AAG,AAGAGAGAGAGAG,A',
 'PASS',
 'lcr\n',
 'AC=1823,921,4177,758,133,1801',
 'AF=6.56180e-02,3.31510e-02,1.50349e-01,2.72839e-02,4.78727e-03,6.48261e-02',
 'AN=27782',
 'AC_AFR=78,90,320,61,32,43',
 'AC_AMR=112,77,394,74,11,83',
 'AC_ASJ=25,20,69,16,0,18',
 'AC_EAS=98,189,343,106,42,18',
 'AC_FIN=342,124,703,117,5,427',
 'AC_NFE=1021,346,1983,315,31,1033',
 'AC_OTH=32,20,79,12,2,28',
 'AC_SAS=115,55,286,57,10,151',
 'AC_Male=1007,490,2308,429,73,1019',
 'AC_Female=816,431,1869,329,60,782',
 'AN_AFR=2136',
 'AN_AMR=2198',
 'AN_ASJ=452',
 'AN_EAS=1246',
 'AN_FIN=4478',
 'AN_NFE=15044',
 'AN_OTH=526',
 'AN_SAS=1702',
 'AN_Male=15080',
 'AN_Female=12702',
 'AF_AFR=3.65169e-02,4.21348e-02,1.49813e-01,2.85581e-02,1.49813e-02,2.01311e-02',
 'AF_AMR=5.09554e-02,3.50318e-02,1.79254e-01,3.36670e-02,5.00455e-03,3.77616e-02',
 'AF_ASJ=5.53097e-02,4.42478e-02,1.52655e-01,3.53982e-02,0.00000e+00,3.98230e-02',
 'AF_EAS=7.86517e-02,1.51685e-01,2.7

In [8]:
re.split("=|,", gnomad['5:172745269'][6])[1]

'6.56180e-02'

1) Match keys between the dictionaries
2) ensure the A1 and A1 are the same
    - gnomad has multiple variants per list -- how best to separate these?
3) Create Fisher's exact table for testing
    - case AC, case AN-AC, control AC, control AN-AC

In [9]:
gnomad_Set = set(gnomad)
optom_Set = set(optom)

results = {}
results_gws = {}
bad_snps = {}
for name in optom_Set.intersection(gnomad_Set):
    if optom[name][1] == gnomad[name][1] and optom[name][2] == gnomad[name][2]:
        if (re.split("=|,", gnomad[name][6])[1] != ".") and (float(re.split("=|,", gnomad[name][6])[1]) <= 0.05):
            #print(optom[name][1], gnomad[name][1], optom[name][2], gnomad[name][2])
            case_AC = int(re.split("=|,", optom[name][4])[1])
            case_AN = int(optom[name][6].split('=')[1])
            control_AC = int(re.split("=|,", gnomad[name][5])[1])
            control_AN = int(gnomad[name][7].split('=')[1])
            #print(case_AC, case_AN, control_AC, control_AN)
            oddsratio, pvalue = stats.fisher_exact([[case_AC, case_AN-case_AC], [control_AC, control_AN-control_AC]], alternative='two-sided')
            info = [case_AC, case_AN, control_AC, control_AN, gnomad[name][3], re.split("=|,", gnomad[name][6])[1], oddsratio, pvalue, gnomad[name][4]]
            if name not in results:
                results[name] = info
            if pvalue < 0.000005 :
                if name not in results_gws:
                    results_gws[name] = info
    else:
        data = [optom[name][1],  optom[name][2], gnomad[name][1], gnomad[name][2]]
        bad_snps[name] = data
        

In [10]:
len(bad_snps)

53939

In [11]:
bad_snps_still = {}

for name in bad_snps:
    if optom[name][1] == gnomad[name][1] and optom[name][2] == gnomad[name][2].split(',')[0] :
        if (re.split("=|,", gnomad[name][6])[1] != ".") and (float(re.split("=|,", gnomad[name][6])[1]) <= 0.05):
            #print(optom[name][1], gnomad[name][1], optom[name][2], gnomad[name][2])
            case_AC = int(re.split("=|,", optom[name][4])[1])
            case_AN = int(optom[name][6].split('=')[1])
            control_AC = int(re.split("=|,", gnomad[name][5])[1])
            control_AN = int(gnomad[name][7].split('=')[1])
            #print(case_AC, case_AN, control_AC, control_AN)
            oddsratio, pvalue = stats.fisher_exact([[case_AC, case_AN-case_AC], [control_AC, control_AN-control_AC]], alternative='two-sided')
            info = [case_AC, case_AN, control_AC, control_AN, gnomad[name][3], re.split("=|,", gnomad[name][6])[1], oddsratio, pvalue, gnomad[name][4]]
            if name not in results:
                results[name] = info
            if pvalue < 0.000005 :
                if name not in results_gws:
                    results_gws[name] = info
    elif len(re.split("=|,", optom[name][3])) > 2:
        if optom[name][1] == gnomad[name][1] and optom[name][2].split(',')[0] == gnomad[name][2].split(',')[0] :
            if (re.split("=|,", gnomad[name][6])[1] != ".") and (float(re.split("=|,", gnomad[name][6])[1]) <= 0.05):
                #print(optom[name][1], gnomad[name][1], optom[name][2], gnomad[name][2])
                case_AC = int(re.split("=|,", optom[name][4])[1])
                case_AN = int(optom[name][6].split('=')[1])
                control_AC = int(re.split("=|,", gnomad[name][5])[1])
                control_AN = int(gnomad[name][7].split('=')[1])
                #print(case_AC, case_AN, control_AC, control_AN)
                oddsratio, pvalue = stats.fisher_exact([[case_AC, case_AN-case_AC], [control_AC, control_AN-control_AC]], alternative='two-sided')
                info = [case_AC, case_AN, control_AC, control_AN, gnomad[name][3], re.split("=|,", gnomad[name][6])[1], oddsratio, pvalue, gnomad[name][4]]
                new_name  = name + "_1"
                if new_name not in results:
                    results[new_name] = info
                if pvalue < 0.000005 :
                    if new_name not in results_gws:
                        results_gws[new_name] = info
    elif len(re.split("=|,", optom[name][3])) > 3:
        if optom[name][1] == gnomad[name][1] and optom[name][2].split(',')[1] == gnomad[name][2].split(',')[1] :
            if (re.split("=|,", gnomad[name][6])[1] != ".") and (float(re.split("=|,", gnomad[name][6])[2]) <= 0.05):
                #print(optom[name][1], gnomad[name][1], optom[name][2], gnomad[name][2])
                case_AC = int(re.split("=|,", optom[name][4])[2])
                case_AN = int(optom[name][6].split('=')[1])
                control_AC = int(re.split("=|,", gnomad[name][5])[2])
                control_AN = int(gnomad[name][7].split('=')[1])
                #print(case_AC, case_AN, control_AC, control_AN)
                oddsratio, pvalue = stats.fisher_exact([[case_AC, case_AN-case_AC], [control_AC, control_AN-control_AC]], alternative='two-sided')
                info = [case_AC, case_AN, control_AC, control_AN, gnomad[name][3], re.split("=|,", gnomad[name][6])[2], oddsratio, pvalue, gnomad[name][4]]
                new_name  = name + "_2"
                if new_name not in results:
                    results[new_name] = info
                if pvalue < 0.000005 :
                    if new_name not in results_gws:
                        results_gws[new_name] = info   
    elif len(re.split("=|,", optom[name][3])) > 4:
        if optom[name][1] == gnomad[name][1] and optom[name][2].split(',')[2] == gnomad[name][2].split(',')[2] :
            if (re.split("=|,", gnomad[name][6])[1] != ".") and (float(re.split("=|,", gnomad[name][6])[3]) <= 0.05):
                #print(optom[name][1], gnomad[name][1], optom[name][2], gnomad[name][2])
                case_AC = int(re.split("=|,", optom[name][4])[3])
                case_AN = int(optom[name][6].split('=')[1])
                control_AC = int(re.split("=|,", gnomad[name][5])[3])
                control_AN = int(gnomad[name][7].split('=')[1])
                #print(case_AC, case_AN, control_AC, control_AN)
                oddsratio, pvalue = stats.fisher_exact([[case_AC, case_AN-case_AC], [control_AC, control_AN-control_AC]], alternative='two-sided')
                info = [case_AC, case_AN, control_AC, control_AN, gnomad[name][3], re.split("=|,", gnomad[name][6])[3], oddsratio, pvalue, gnomad[name][4]]
                new_name  = name + "_3"
                if new_name not in results:
                    results[new_name] = info
                if pvalue < 0.000005 :
                    if new_name not in results_gws:
                        results_gws[new_name] = info   
    else: 
        data = [optom[name][1],  optom[name][2], gnomad[name][1], gnomad[name][2]]
        bad_snps_still[name] = data

In [12]:
len(bad_snps_still)

14070

In [13]:
len(results)

79832

In [30]:
all_keys = set(list(results.keys()) + list(optom.keys()))

optom_sig = {}
for key in all_keys:
    if key in results and key in optom:
        optom_sig[key] = optom[key] + results[key]

In [31]:
test_sig = pd.DataFrame(optom_sig).T.reset_index()
test_sig.rename(columns={'index': 'chr:bp', 0: 'rs_id', 1:'A1', 2:'A2', 3:'flag_optom', 4:'AC', 5:'AF', 6:'AN', 
                   7:'filter',  8:'case_AC', 9:'case_AN', 10:'control_AC', 11:'control_AN', 
                   12:'flag_gnomad', 13:'gnomad_AF', 14:'oddsratio', 15:'pvalue', 16:'annotations'}, inplace=True)
test_sig.head()

,chr:bp,rs_id,A1,A2,flag_optom,AC,AF,AN,filter,case_AC,case_AN,control_AC,control_AN,flag_gnomad,gnomad_AF,oddsratio,pvalue,annotations
0,10:100011298,rs138062420,G,A,PASS,AC=0,AF=0.013,AN=30,BaseQRankSum=-2.492e+00,0,30,602,243964,PASS,2.46758e-03,0,1,CSQ=A|intron_variant|MODIFIER|LOXL4|ENSG000001...
1,10:100144676,rs191156429,C,T,PASS,AC=0,AF=0.013,AN=30,BaseQRankSum=-6.391e+00,0,30,66,225918,PASS,2.92141e-04,0,1,CSQ=T|intron_variant|MODIFIER|PYROXD2|ENSG0000...
2,10:100148117,rs147555747,C,T,PASS,AC=1,AF=0.013,AN=30,BaseQRankSum=-2.965e+00,1,30,435,245902,PASS,1.76900e-03,19.4583,0.0518436,CSQ=T|missense_variant|MODERATE|PYROXD2|ENSG00...
3,10:100150830,.,G,A,PASS,AC=0,AF=0.013,AN=30,BaseQRankSum=-2.322e+00,0,30,246,243128,RF,1.01181e-03,0,1,CSQ=A|missense_variant|MODERATE|PYROXD2|ENSG00...
4,10:100167323,rs79218397,G,A,PASS,AC=1,AF=0.013,AN=30,BaseQRankSum=1.96,1,30,1012,243762,PASS,4.15159e-03,8.27143,0.117435,CSQ=A|intron_variant|MODIFIER|PYROXD2|ENSG0000...


In [32]:
len(test_sig)

79832

In [33]:
test_sig.to_csv('optom_results_exac.csv', index=False, header=True, sep='\t')

In [34]:
all_keys = set(list(results.keys()) + list(optom.keys()))

optom_sig_gws = {}
for key in all_keys:
    if key in results_gws and key in optom:
        optom_sig_gws[key] = optom[key] + results_gws[key]

In [35]:
len(optom_sig_gws)

2545

In [37]:
test = pd.DataFrame(optom_sig_gws).T.reset_index()
test.rename(columns={'index': 'chr:bp', 0: 'rs_id', 1:'A1', 2:'A2', 3:'flag_optom', 4:'AC', 5:'AF', 6:'AN', 
                   7:'filter',  8:'case_AC', 9:'case_AN', 10:'control_AC', 11:'control_AN', 
                   12:'flag_gnomad', 13:'gnomad_AF', 14:'oddsratio', 15:'pvalue', 16:'annotations'}, inplace=True)
test.head()

,chr:bp,rs_id,A1,A2,flag_optom,AC,AF,AN,filter,case_AC,case_AN,control_AC,control_AN,flag_gnomad,gnomad_AF,oddsratio,pvalue,annotations
0,10:116698112,rs199739741,A,C,VQSRTrancheSNP99.90to100.00,AC=9,AF=0.113,AN=30,BaseQRankSum=-3.965e+00,9,30,3424,162722,RF,2.10420e-02,19.9388,7.84166e-09,CSQ=C|missense_variant|MODERATE|TRUB1|ENSG0000...
1,10:118927198,.,TGGTCCCGGCGCCCGGCCTGTGTGGTGCTCAGATCAGACTTATCCA...,T,PASS,AC=8,AF=0.100,AN=30,BaseQRankSum=-1.360e-01,8,30,89,6468,RF,1.37600e-02,26.0633,8.22806e-09,CSQ=-|intron_variant&non_coding_transcript_var...
2,10:135438959,.,AC,A,VQSRTrancheINDEL99.90to100.00,AC=6,AF=0.225,AN=30,BaseQRankSum=-3.050e-01,6,30,1,244284,PASS,4.09360e-06,61070.8,1.40719e-23,segdup\n
3,10:135439015,rs75470891,T,C,VQSRTrancheSNP99.90to100.00,AC=4,AF=0.075,AN=30,BaseQRankSum=-6.700e-02,4,30,729,238948,RF,3.05087e-03,50.2731,2.25792e-06,segdup\n
4,10:135439049,rs200347477,G,C,VQSRTrancheSNP99.90to100.00,AC=5,AF=0.113,AN=30,BaseQRankSum=1.89,5,30,360,227178,RF,1.58466e-03,126.01,1.4352e-09,segdup\n


In [38]:
test.sort_values(by="pvalue").head()

,chr:bp,rs_id,A1,A2,flag_optom,AC,AF,AN,filter,case_AC,case_AN,control_AC,control_AN,flag_gnomad,gnomad_AF,oddsratio,pvalue,annotations
785,16:33962400,rs74410326,C,A,VQSRTrancheSNP99.90to100.00,AC=30,AF=0.988,AN=30,DB,30,30,0,69228,RF;AC0,0.00000e+00,inf,1.63038e-113,segdup\n
784,16:33962391,rs77260646,C,A,VQSRTrancheSNP99.90to100.00,AC=30,AF=1.00,AN=30,DB,30,30,6,70844,RF,8.46931e-05,inf,1.5895e-107,segdup\n
1628,6:32486358,rs115773383,T,C,VQSRTrancheSNP99.90to100.00,AC=28,AF=0.800,AN=30,BaseQRankSum=3.33,28,30,1,68654,PASS,1.45658e-05,961142,1.43038e-102,CSQ=C|synonymous_variant|LOW|HLA-DRB5|ENSG0000...
1512,4:191003163,.,G,T,VQSRTrancheSNP99.00to99.90,AC=30,AF=1.00,AN=30,DP=23,30,30,0,24938,RF;AC0,0.00000e+00,inf,3.23383e-100,segdup\n
1835,6:32610893,rs9272918,C,A,VQSRTrancheSNP99.90to100.00,AC=24,AF=0.782,AN=30,BaseQRankSum=-1.208e+00,24,30,0,76970,RF;AC0,0.00000e+00,inf,1.95933e-88,CSQ=A|3_prime_UTR_variant|MODIFIER|HLA-DQA1|EN...


In [39]:
test.to_csv('optom_gws_sig_results_exac.csv', index=False, header=True, sep='\t')

## Grab optom genotype info for these variants

What do I want:
    1) Number of Hets
    2) Number of Homo for A2
    3) Number of lowGQ

In [40]:
with open("recalibrated_variants.postCGP.Gfiltered.Optom_subset.vcf", 'rt') as f:
    optom_geno = {}
    info = []
    # create a list of locations in optom
    for line in f:
        if line.startswith('#'):
            continue
        row = line.split('\t')
        tmp = [stuff.replace("chr", "") for stuff in row]
        ### create an index which is chr:bp
        chr_bp = tmp[0:2]
        index = ':'.join(map(str, chr_bp))
        if index in optom_sig:
            if index not in optom_geno:
                homo_major = 0
                hets = 0
                homo_minor = 0
                lowGQ_count = 0
                for x in range(9,24) :
                    data = tmp[x].split(':')
                    if data[0] == "0/0" :
                        homo_major += 1
                    elif data[0] == "0/1" :
                        hets += 1
                    elif data[0] == "1/1":
                        homo_minor += 1
                    if data[3] == 'lowGQ':
                        lowGQ_count += 1
                info = [tmp[2], tmp[3], tmp[4], homo_major, hets, homo_minor, lowGQ_count]
                optom_geno[index] = info 

In [41]:
genos = pd.DataFrame(optom_geno).T.reset_index()
genos.rename(columns={'index': 'chr:bp', 0: 'rs_id', 1:'A1', 2:'A2', 3:'homo_major', 4:'hets', 
                      5:'homo_minor', 6:'lowGQ_numbers' }, inplace=True)
genos.head()

,chr:bp,rs_id,A1,A2,homo_major,hets,homo_minor,lowGQ_numbers
0,10:100011298,rs138062420,G,A,15,0,0,0
1,10:100144676,rs191156429,C,T,15,0,0,0
2,10:100148117,rs147555747,C,T,14,1,0,0
3,10:100150830,.,G,A,15,0,0,0
4,10:100167323,rs79218397,G,A,14,1,0,0


In [42]:
genos.to_csv('optom_genotpyes_results_exac.csv', index=False, header=True, sep='\t')

# Association with just EUR population in GNOMAD

In [43]:
# Allele counts to keep
list_names =["AC_NFE", "AN_NFE", "AF_NFE"]
pattern = re.compile("|".join(list_names))

#not_pass_qc_gnomad = {}

with open("gnomad.exomes.r2.0.2.sites.vcf", 'rt') as vcf_nfe:
    gnomad_NFE = {}
    for line in vcf_nfe:
        if line.startswith('#'):
            continue 
        row = line.split('\t')
        ### create an index which is chr:bp
        chr_bp = row[0:2]
        index = ':'.join(map(str, chr_bp))
        ### keep only those that are in optom:
        if index in optom:
            #if row[6] != "PASS":
            #    if index not in not_pass_qc_gnomad:
            #        not_pass_qc_gnomad[index] = [ row[2], row[6] ]
            #    continue
            ### output index, rsnumber, a1, a2, info
            output = [ row[2], row[3], row[4], row[6], row[-1].split(';')[-1] ]
            info = row[7].split(';')
            keep_info = [item for item in info if re.match(pattern, item)]
            output.extend(keep_info)
            #print(info)
            if index not in gnomad_NFE:
                gnomad_NFE[index] = output
        elif row[2] in snp:
            if index not in gnomad_NFE:
                ### output index, rsnumber, a1, a2, info
                output = [ index, row[2], row[3], row[4], row[6], row[-1].split(';')[-1] ]
                info = row[7].split(';')
                keep_info = [item for item in info if re.match(pattern, item)]
                output.extend(keep_info)
                gnomad_NFE[index] = output

In [44]:
len(gnomad_NFE)

188436

In [45]:
gnomad_NFE['5:172745269']

['rs4041247',
 'AAGAGAG',
 'AAGAGAGAG,AAGAGAGAGAG,AAGAG,AAG,AAGAGAGAGAGAG,A',
 'PASS',
 'lcr\n',
 'AC_NFE=1021,346,1983,315,31,1033',
 'AN_NFE=15044',
 'AF_NFE=6.78676e-02,2.29992e-02,1.31813e-01,2.09386e-02,2.06062e-03,6.86652e-02']

In [46]:
gnomad_Set = set(gnomad_NFE)
optom_Set = set(optom)

results_NFE = {}
results_gws_NFE = {}
bad_snps = {}
for name in optom_Set.intersection(gnomad_Set):
    if optom[name][1] == gnomad_NFE[name][1] and optom[name][2] == gnomad_NFE[name][2]:
        if (re.split("=|,", gnomad_NFE[name][7])[1] != ".") and (float(re.split("=|,", gnomad_NFE[name][7])[1]) <= 0.05):
            #print(optom[name][1], gnomad[name][1], optom[name][2], gnomad[name][2])
            case_AC = int(re.split("=|,", optom[name][4])[1])
            case_AN = int(optom[name][6].split('=')[1])
            control_AC = int(re.split("=|,", gnomad_NFE[name][5])[1])
            control_AN = int(gnomad_NFE[name][6].split('=')[1])
            #print(case_AC, case_AN, control_AC, control_AN)
            oddsratio, pvalue = stats.fisher_exact([[case_AC, case_AN-case_AC], [control_AC, control_AN-control_AC]], alternative='two-sided')
            info = [case_AC, case_AN, control_AC, control_AN, gnomad_NFE[name][3], re.split("=|,", gnomad_NFE[name][7])[1], oddsratio, pvalue, gnomad_NFE[name][4]]
            if name not in results_NFE:
                results_NFE[name] = info
            if pvalue < 0.000005 :
                if name not in results_gws_NFE:
                    results_gws_NFE[name] = info
    else:
        data = [optom[name][1],  optom[name][2], gnomad_NFE[name][1], gnomad_NFE[name][2]]
        bad_snps[name] = data

In [47]:
len(bad_snps)

53939

In [48]:
bad_snps_still = {}

for name in bad_snps:
    if optom[name][1] == gnomad_NFE[name][1] and optom[name][2] == gnomad_NFE[name][2].split(',')[0] :
        if (re.split("=|,", gnomad_NFE[name][7])[1] != ".") and (float(re.split("=|,", gnomad_NFE[name][7])[1]) <= 0.05):
            #print(optom[name][1], gnomad[name][1], optom[name][2], gnomad[name][2])
            case_AC = int(re.split("=|,", optom[name][4])[1])
            case_AN = int(optom[name][6].split('=')[1])
            control_AC = int(re.split("=|,", gnomad_NFE[name][5])[1])
            control_AN = int(gnomad_NFE[name][6].split('=')[1])
            #print(case_AC, case_AN, control_AC, control_AN)
            oddsratio, pvalue = stats.fisher_exact([[case_AC, case_AN-case_AC], [control_AC, control_AN-control_AC]], alternative='two-sided')
            info = [case_AC, case_AN, control_AC, control_AN, gnomad_NFE[name][3], re.split("=|,", gnomad_NFE[name][7])[1], oddsratio, pvalue, gnomad_NFE[name][4]]
            if name not in results_NFE:
                results_NFE[name] = info
            if pvalue < 0.000005 :
                if name not in results_gws_NFE:
                    results_gws_NFE[name] = info
    elif len(re.split("=|,", optom[name][3])) > 2:
        if optom[name][1] == gnomad_NFE[name][1] and optom[name][2].split(',')[0] == gnomad_NFE[name][2].split(',')[0] :
            if (re.split("=|,", gnomad_NFE[name][7])[1] != ".") and (float(re.split("=|,", gnomad_NFE[name][7])[1]) <= 0.05):
                #print(optom[name][1], gnomad[name][1], optom[name][2], gnomad[name][2])
                case_AC = int(re.split("=|,", optom[name][4])[1])
                case_AN = int(optom[name][6].split('=')[1])
                control_AC = int(re.split("=|,", gnomad_NFE[name][5])[1])
                control_AN = int(gnomad_NFE[name][6].split('=')[1])
                #print(case_AC, case_AN, control_AC, control_AN)
                oddsratio, pvalue = stats.fisher_exact([[case_AC, case_AN-case_AC], [control_AC, control_AN-control_AC]], alternative='two-sided')
                info = [case_AC, case_AN, control_AC, control_AN, gnomad_NFE[name][3], re.split("=|,", gnomad_NFE[name][7])[1], oddsratio, pvalue, gnomad_NFE[name][4]]
                new_name = name + "_1"
                if new_name not in results_NFE:
                    results_NFE[new_name] = info
                if pvalue < 0.000005 :
                    if new_name not in results_gws_NFE:
                        results_gws_NFE[new_name] = info
    elif len(re.split("=|,", optom[name][3])) > 3:
        if optom[name][1] == gnomad_NFE[name][1] and optom[name][2].split(',')[1] == gnomad_NFE[name][2].split(',')[1] :
            if (re.split("=|,", gnomad_NFE[name][7])[1] != ".") and (float(re.split("=|,", gnomad_NFE[name][7])[2]) <= 0.05):
                #print(optom[name][1], gnomad[name][1], optom[name][2], gnomad[name][2])
                case_AC = int(re.split("=|,", optom[name][4])[2])
                case_AN = int(optom[name][6].split('=')[1])
                control_AC = int(re.split("=|,", gnomad_NFE[name][5])[2])
                control_AN = int(gnomad_NFE[name][6].split('=')[1])
                #print(case_AC, case_AN, control_AC, control_AN)
                oddsratio, pvalue = stats.fisher_exact([[case_AC, case_AN-case_AC], [control_AC, control_AN-control_AC]], alternative='two-sided')
                info = [case_AC, case_AN, control_AC, control_AN, gnomad_NFE[name][3], re.split("=|,", gnomad_NFE[name][7])[2], oddsratio, pvalue, gnomad_NFE[name][4]]
                new_name = name + "_2"
                if new_name not in results_NFE:
                    results_NFE[new_name] = info
                if pvalue < 0.000005 :
                    if new_name not in results_gws_NFE:
                        results_gws_NFE[new_name] = info   
    elif len(re.split("=|,", optom[name][3])) > 4:
        if optom[name][1] == gnomad_NFE[name][1] and optom[name][2].split(',')[2] == gnomad_NFE[name][2].split(',')[2] :
            if (re.split("=|,", gnomad_NFE[name][7])[1] != ".") and (float(re.split("=|,", gnomad_NFE[name][7])[3]) <= 0.05):
                #print(optom[name][1], gnomad[name][1], optom[name][2], gnomad[name][2])
                case_AC = int(re.split("=|,", optom[name][4])[3])
                case_AN = int(optom[name][6].split('=')[1])
                control_AC = int(re.split("=|,", gnomad_NFE[name][5])[3])
                control_AN = int(gnomad_NFE[name][6].split('=')[1])
                #print(case_AC, case_AN, control_AC, control_AN)
                oddsratio, pvalue = stats.fisher_exact([[case_AC, case_AN-case_AC], [control_AC, control_AN-control_AC]], alternative='two-sided')
                info = [case_AC, case_AN, control_AC, control_AN, gnomad_NFE[name][3], re.split("=|,", gnomad_NFE[name][7])[3], oddsratio, pvalue, gnomad_NFE[name][4]]
                new_name = name + "_3"
                if new_name not in results_NFE:
                    results_NFE[new_name] = info
                if pvalue < 0.000005 :
                    if new_name not in results_gws_NFE:
                        results_gws_NFE[new_name] = info  
    else: 
        data = [optom[name][1],  optom[name][2], gnomad_NFE[name][1], gnomad_NFE[name][2]]
        bad_snps_still[name] = data

In [49]:
len(bad_snps_still)

14070

In [50]:
all_keys = set(list(results_NFE.keys()) + list(optom.keys()))

optom_sig_NFE = {}
for key in all_keys:
    if key in results_NFE and key in optom:
        optom_sig_NFE[key] = optom[key] + results_NFE[key]

In [51]:
test_NFE_sig = pd.DataFrame(optom_sig_NFE).T.reset_index()
test_NFE_sig.rename(columns={'index': 'chr:bp', 0: 'rs_id', 1:'A1', 2:'A2', 3:'flag_optom', 4:'AC', 5:'AF', 6:'AN', 
                   7:'filter',  8:'case_AC', 9:'case_AN', 10:'control_AC', 11:'control_AN', 
                   12:'flag_gnomad', 13:'gnomad_AF', 14:'oddsratio', 15:'pvalue', 16:'annotations'}, inplace=True)
test_NFE_sig.head()

,chr:bp,rs_id,A1,A2,flag_optom,AC,AF,AN,filter,case_AC,case_AN,control_AC,control_AN,flag_gnomad,gnomad_AF,oddsratio,pvalue,annotations
0,10:100011298,rs138062420,G,A,PASS,AC=0,AF=0.013,AN=30,BaseQRankSum=-2.492e+00,0,30,234,110236,PASS,2.12272e-03,0,1,CSQ=A|intron_variant|MODIFIER|LOXL4|ENSG000001...
1,10:100144676,rs191156429,C,T,PASS,AC=0,AF=0.013,AN=30,BaseQRankSum=-6.391e+00,0,30,61,104180,PASS,5.85525e-04,0,1,CSQ=T|intron_variant|MODIFIER|PYROXD2|ENSG0000...
2,10:100148117,rs147555747,C,T,PASS,AC=1,AF=0.013,AN=30,BaseQRankSum=-2.965e+00,1,30,360,111504,PASS,3.22858e-03,10.646,0.0926895,CSQ=T|missense_variant|MODERATE|PYROXD2|ENSG00...
3,10:100150830,.,G,A,PASS,AC=0,AF=0.013,AN=30,BaseQRankSum=-2.322e+00,0,30,121,110074,RF,1.09926e-03,0,1,CSQ=A|missense_variant|MODERATE|PYROXD2|ENSG00...
4,10:100167323,rs79218397,G,A,PASS,AC=1,AF=0.013,AN=30,BaseQRankSum=1.96,1,30,27,110718,PASS,2.43863e-04,141.368,0.00755803,CSQ=A|intron_variant|MODIFIER|PYROXD2|ENSG0000...


In [53]:
test_NFE_sig.to_csv('optom_results_exac_NFE.csv', index=False, header=True, sep='\t')

In [54]:
all_keys = set(list(results_gws_NFE.keys()) + list(optom.keys()))

optom_sig_gws_NFE = {}
for key in all_keys:
    if key in results_gws_NFE and key in optom:
        optom_sig_gws_NFE[key] = optom[key] + results_gws_NFE[key]

In [55]:
len(optom_sig_gws_NFE)

2539

In [56]:
test_NFE = pd.DataFrame(optom_sig_gws_NFE).T.reset_index()
test_NFE.rename(columns={'index': 'chr:bp', 0: 'rs_id', 1:'A1', 2:'A2', 3:'flag_optom', 4:'AC', 5:'AF', 6:'AN', 
                   7:'filter',  8:'case_AC', 9:'case_AN', 10:'control_AC', 11:'control_AN', 
                   12:'flag_gnomad', 13:'gnomad_AF', 14:'oddsratio', 15:'pvalue', 16:'annotations'}, inplace=True)
test_NFE.head()

,chr:bp,rs_id,A1,A2,flag_optom,AC,AF,AN,filter,case_AC,case_AN,control_AC,control_AN,flag_gnomad,gnomad_AF,oddsratio,pvalue,annotations
0,10:116698112,rs199739741,A,C,VQSRTrancheSNP99.90to100.00,AC=9,AF=0.113,AN=30,BaseQRankSum=-3.965e+00,9,30,1572,69330,RF,2.26742e-02,18.4727,1.50896e-08,CSQ=C|missense_variant|MODERATE|TRUB1|ENSG0000...
1,10:118927198,.,TGGTCCCGGCGCCCGGCCTGTGTGGTGCTCAGATCAGACTTATCCA...,T,PASS,AC=8,AF=0.100,AN=30,BaseQRankSum=-1.360e-01,8,30,74,5516,RF,1.34155e-02,26.742,7.26155e-09,CSQ=-|intron_variant&non_coding_transcript_var...
2,10:135438959,.,AC,A,VQSRTrancheINDEL99.90to100.00,AC=6,AF=0.225,AN=30,BaseQRankSum=-3.050e-01,6,30,0,111166,PASS,0.00000e+00,inf,2.26192e-22,segdup\n
3,10:135439015,rs75470891,T,C,VQSRTrancheSNP99.90to100.00,AC=4,AF=0.075,AN=30,BaseQRankSum=-6.700e-02,4,30,201,109326,RF,1.83854e-03,83.5247,3.16307e-07,segdup\n
4,10:135439049,rs200347477,G,C,VQSRTrancheSNP99.90to100.00,AC=5,AF=0.113,AN=30,BaseQRankSum=1.89,5,30,115,104664,RF,1.09875e-03,181.824,2.53267e-10,segdup\n


In [57]:
test_NFE.sort_values(by="pvalue").head()

,chr:bp,rs_id,A1,A2,flag_optom,AC,AF,AN,filter,case_AC,case_AN,control_AC,control_AN,flag_gnomad,gnomad_AF,oddsratio,pvalue,annotations
787,16:33962400,rs74410326,C,A,VQSRTrancheSNP99.90to100.00,AC=30,AF=0.988,AN=30,DB,30,30,0,23734,RF;AC0,0.00000e+00,inf,1.4257e-99,segdup\n
786,16:33962391,rs77260646,C,A,VQSRTrancheSNP99.90to100.00,AC=30,AF=1.00,AN=30,DB,30,30,0,23358,RF,0.00000e+00,inf,2.30117e-99,segdup\n
1611,6:32486358,rs115773383,T,C,VQSRTrancheSNP99.90to100.00,AC=28,AF=0.800,AN=30,BaseQRankSum=3.33,28,30,0,32194,PASS,0.00000e+00,inf,7.91957e-95,CSQ=C|synonymous_variant|LOW|HLA-DRB5|ENSG0000...
1494,4:191003163,.,G,T,VQSRTrancheSNP99.00to99.90,AC=30,AF=1.00,AN=30,DP=23,30,30,0,8014,RF;AC0,0.00000e+00,inf,1.91872e-85,segdup\n
1822,6:32610893,rs9272918,C,A,VQSRTrancheSNP99.90to100.00,AC=24,AF=0.782,AN=30,BaseQRankSum=-1.208e+00,24,30,0,30596,RF;AC0,0.00000e+00,inf,8.01818e-79,CSQ=A|3_prime_UTR_variant|MODIFIER|HLA-DQA1|EN...


In [58]:
test_NFE.to_csv('optom_gws_sig_results_exac_NFE.csv', index=False, header=True, sep='\t')

## Grab optom genotype info for these variants

In [59]:
with open("recalibrated_variants.postCGP.Gfiltered.Optom_subset.vcf", 'rt') as f:
    optom_geno_NFE = {}
    info = []
    # create a list of locations in optom
    for line in f:
        if line.startswith('#'):
            continue
        row = line.split('\t')
        tmp = [stuff.replace("chr", "") for stuff in row]
        ### create an index which is chr:bp
        chr_bp = tmp[0:2]
        index = ':'.join(map(str, chr_bp))
        if index in optom_sig_NFE:
            if index not in optom_geno_NFE:
                homo_major = 0
                hets = 0
                homo_minor = 0
                lowGQ_count = 0
                for x in range(9,24) :
                    data = tmp[x].split(':')
                    if data[0] == "0/0" :
                        homo_major += 1
                    elif data[0] == "0/1" :
                        hets += 1
                    elif data[0] == "1/1":
                        homo_minor += 1
                    if data[3] == 'lowGQ':
                        lowGQ_count += 1
                info = [tmp[2], tmp[3], tmp[4], homo_major, hets, homo_minor, lowGQ_count]
                optom_geno_NFE[index] = info 

In [60]:
genos_NFE = pd.DataFrame(optom_geno_NFE).T.reset_index()
genos_NFE.rename(columns={'index': 'chr:bp', 0: 'rs_id', 1:'A1', 2:'A2', 3:'homo_major', 4:'hets', 
                      5:'homo_minor', 6:'lowGQ_numbers'}, inplace=True)
genos_NFE.head()

,chr:bp,rs_id,A1,A2,homo_major,hets,homo_minor,lowGQ_numbers
0,10:100011298,rs138062420,G,A,15,0,0,0
1,10:100144676,rs191156429,C,T,15,0,0,0
2,10:100148117,rs147555747,C,T,14,1,0,0
3,10:100150830,.,G,A,15,0,0,0
4,10:100167323,rs79218397,G,A,14,1,0,0


In [61]:
genos_NFE.to_csv('optom_genotpyes_results_exac_NFE.csv', index=False, header=True, sep='\t')